<a href="https://colab.research.google.com/github/SSAC-AI/resnet/blob/main/Bottleneck_Block.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

| 인자                        | 설명                                |
| ------------------------- | --------------------------------- |
| `in_channels`             | 입력 텐서의 채널 수                       |
| `out_channels`            | 출력 채널 수 (shortcut 포함 최종 출력)       |
| `bottleneck_channels`     | 중간 bottleneck 레이어의 채널 수 (작은 값)    |
| `stride`                  | 다운샘플링 여부 (1 또는 2)                 |
| `dilation`                | Atrous convolution을 위한 dilation 값 |
| `use_bounded_activations` | ReLU6와 clip 연산을 사용할지 여부 (양자화에 유리) |


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Bottleneck(nn.Module):
    """
    ResNet의 Bottleneck 블록입니다.
    각 블록은 1x1 -> 3x3 -> 1x1 Conv 레이어로 구성되어 있습니다.
    shortcut 연결을 통해 입력과 출력의 skip connection을 수행합니다.
    """
    expansion = 4  # 출력 채널 수를 확장하는 비율

    def __init__(self, in_channels, bottleneck_channels, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, bottleneck_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(bottleneck_channels)

        self.conv2 = nn.Conv2d(
            bottleneck_channels,
            bottleneck_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False
        )
        self.bn2 = nn.BatchNorm2d(bottleneck_channels)

        self.conv3 = nn.Conv2d(
            bottleneck_channels,
            bottleneck_channels * self.expansion,
            kernel_size=1,
            bias=False
        )
        self.bn3 = nn.BatchNorm2d(bottleneck_channels * self.expansion)

        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample  # 입력 채널과 출력 채널이 다를 경우 사용

    def forward(self, x):
        identity = x

        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    """
    ResNet의 전체 구조입니다.
    ResNet-50, 101, 152, 200과 같이 블록 수를 조절하여 다양한 깊이의 모델을 생성할 수 있습니다.
    """

    def __init__(self, block, layers, num_classes=1000):
        super(ResNet, self).__init__()
        self.in_channels = 64

        # 초기 stem: conv -> BN -> ReLU -> maxpool
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # ResNet 블록 스택 (4개 스테이지)
        self.layer1 = self._make_layer(block, 64, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        # 글로벌 평균 풀링 후 FC 레이어
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride):
        """
        특정 스테이지에서 여러 개의 Bottleneck 블록을 쌓는 함수
        """
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    out_channels * block.expansion,
                    kernel_size=1,
                    stride=stride,
                    bias=False
                ),
                nn.BatchNorm2d(out_channels * block.expansion),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion

        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x


# 모델 생성 함수들: 원하는 depth에 따라 호출

def resnet50(num_classes=1000):
    return ResNet(Bottleneck, [3, 4, 6, 3], num_classes)

def resnet101(num_classes=1000):
    return ResNet(Bottleneck, [3, 4, 23, 3], num_classes)

def resnet152(num_classes=1000):
    return ResNet(Bottleneck, [3, 8, 36, 3], num_classes)

def resnet200(num_classes=1000):
    return ResNet(Bottleneck, [3, 24, 36, 3], num_classes)


# 사용 예시:
# model = resnet50(num_classes=1000)
# input_tensor = torch.randn(1, 3, 224, 224)
# output = model(input_tensor)
# print(output.shape)  # torch.Size([1, 1000])
